Model ResNet

https://www.pluralsight.com/guides/introduction-to-resnet

## TODOS
6. Debugging, does output make sense?
    1. Resize images
    2. preprocessing fixes
    5. replace scaling by proper function
3. navigation
4. controller



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import time
import copy

import os
import sys
sys.path.insert(1, 'C:/Users/morit/OneDrive/UNI/Master/WS22/APP-RAS/Programming/data_pipeline')
import os
from data_sampler import WeightedSampler
from dataset import CARLADataset#, CARLADatasetMultiProcessing
from torch.utils.data import DataLoader

In [4]:
class MyResnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torchvision.models.resnet18(weights=True)
        num_ftrs = self.net.fc.in_features
        """
        self.net.fc = nn.Sequential(
            nn.Linear(512, 3), #nbr of outputs self.net.fc.in_features
            nn.Tanh()
        )
        """
        self.net.fc = nn.Identity()
        
        self.thr_head = nn.Sequential(
            nn.Linear(num_ftrs, 1),
            nn.Sigmoid() # TODO BU
        )
        
        self.brk_head = nn.Sequential(
            nn.Linear(num_ftrs, 1),
            nn.Sigmoid()
        )
        self.str_head = nn.Sequential(
            nn.Linear(num_ftrs, 1),
            nn.Tanh()
        )
        
        self.net = self.net.cuda() if device else self.net
        self.net.fc = self.net.fc.cuda() if device else self.net.fc
        """
        self.thr_head = self.thr_head.cuda() if device else self.thr_head
        self.brk_head = self.brk_head.cuda() if device else self.brk_head
        self.str_head = self.str_head.cuda() if device else self.str_head
        """

    def forward(self, data_):
        x = self.net(data_)
        x = self.net.fc(x)
        return self.thr_head(x), self.str_head(x), self.brk_head(x)

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = MyResnet()
net = net.cuda() if device else net
net

MyResnet(
  (net): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

Data Loaders etc


In [6]:
path_ege_data = os.path.join("..", "data", "Dataset Ege")

config = {"used_inputs": ["rgb","measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake"],
        "seq_len": 1
        }

train_dataset = CARLADataset(root_dir=path_ege_data, config=config, transform=None)
weighted_sampler = WeightedSampler(dataset=train_dataset)

batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

Town10HD_Scenario10_route9_11_28_17_59_39 has varying number of data files among input folders. It got discarded from the dataset.


In [8]:
"""
def normalize_img(tensor):
    v_min, v_max = tensor.min(), tensor.max()
    new_min, new_max = 0, 1
    v_p = (tensor - v_min)/(v_max - v_min)*(new_max - new_min) + new_min
    return v_p
"""

In [41]:
def normalize_batch(tensors):
    liste = []
    for tensor in tensors:
        tensor = preprocess(tensor * 1/255) 
        liste.append(tensor)
    return torch.stack(liste)     

In [42]:
from PIL import Image
from torchvision import transforms
#input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

img = next(iter(train_dataloader))["rgb"].transpose(1,4)[:,:,:,:,0].float()
img.shape

torch.Size([16, 3, 160, 960])

In [44]:
normalize_batch(img).shape#[0,:,:,:]

tensor([[[-0.1999,  0.0056,  0.0741,  ..., -1.6727, -1.6555, -1.7069],
         [-0.1999,  0.0056,  0.0912,  ..., -1.6555, -1.6555, -1.7240],
         [-0.1828,  0.0227,  0.0912,  ..., -1.6727, -1.7069, -1.7583],
         ...,
         [-0.7822, -0.8335, -0.9020,  ..., -0.1143, -0.1143, -0.1314],
         [-0.8507, -0.8849, -0.8849,  ..., -0.1657, -0.1657, -0.1828],
         [-0.9363, -0.9363, -0.8849,  ..., -0.2342, -0.2342, -0.2342]],

        [[-0.0749,  0.0651,  0.1176,  ..., -1.4930, -1.4230, -1.3529],
         [-0.0749,  0.0826,  0.1527,  ..., -1.4930, -1.4580, -1.4230],
         [-0.0574,  0.1001,  0.1702,  ..., -1.4755, -1.4930, -1.5105],
         ...,
         [-0.6877, -0.7402, -0.7402,  ..., -0.3901, -0.3901, -0.3901],
         [-0.7402, -0.7577, -0.7577,  ..., -0.4426, -0.4426, -0.4426],
         [-0.7927, -0.7752, -0.7577,  ..., -0.5126, -0.4951, -0.5126]],

        [[ 0.2173,  0.3393,  0.3742,  ..., -1.2293, -1.1944, -1.1596],
         [ 0.2173,  0.3568,  0.4091,  ..., -1

Training

In [45]:
criterion = nn.L1Loss()#nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

In [46]:
%%time

n_epochs = 1
#print_every = 10
valid_loss_min = np.Inf
val_loss = []
train_loss = []
total_step = len(train_dataloader)

run = True

for epoch in range(1, n_epochs+1):
    
    running_loss = 0.0
    print(f'Epoch {epoch}\n')
    
    #data: (['idx', 'rgb', 'speed', 'steer', 'throttle', 'brake'])
    
    # Work through batches
    for batch_idx, data in enumerate(train_dataloader):

        # further preprocessing
        data_ = normalize_batch(data["rgb"].transpose(1,4)[:,:,:,:,0]).float() # HOTFIX
        #data["rgb"].transpose(1,4)[:,:,:,:,0]
        data["throttle"]=data["throttle"].float()
        data["steer"]=data["steer"].float()
        data["brake"]=data["brake"].float()
        
        #print(target_.shape)
        #print(target_.shape[0])
        if data["throttle"].shape[0] != batch_size: # HOTFIX
            run = False
            break

        # move to GPU
        data_ = data_.to(device)
        
        data["throttle"] = data["throttle"].to(device)
        data["steer"] = data["steer"].to(device)
        data["brake"] = data["brake"].to(device)
        
        # compute outputs
        optimizer.zero_grad()
        
        outputs = net(data_)
        output_throttle = outputs[0].to(device)
        output_steer = outputs[1].to(device)
        output_brake = outputs[2].to(device)
        
        loss_throttle = criterion(output_throttle, data["throttle"].float())
        loss_steer = criterion(output_steer, data["steer"].float())
        loss_brake = criterion(output_brake, data["brake"].float())
        
        
        loss = sum([loss_throttle, loss_steer, loss_brake])/3
        
        # Backprop
        loss.backward()
        
        optimizer.step()

        running_loss += loss.item()
        if (batch_idx) % batch_size == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))

    if not run: # HOTFIX
        continue
        
    # Epoch finished, evaluate network and save if network_learned
    train_loss.append(running_loss/total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f},')
    batch_loss = 0

    
    """
    with torch.no_grad():
        net.eval()
        for data_t, target_t in (test_dataloader):
            data_t, target_t = data_t.to(device), target_t.to(device)
            outputs_t = net(data_t)
            loss_t = criterion(outputs_t, target_t)
            batch_loss += loss_t.item()
        val_loss.append(batch_loss/len(test_dataloader))
        #network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, \n')

        
        if False:#network_learned:
            valid_loss_min = batch_loss
            torch.save(net.state_dict(), 'resnet.pt')
            print('Improvement-Detected, save-model')
    """

    net.train()
    

Epoch 1

Epoch [1/1], Step [0/126], Loss: 0.5484
Epoch [1/1], Step [16/126], Loss: 0.3794
Epoch [1/1], Step [32/126], Loss: 0.4079
Epoch [1/1], Step [48/126], Loss: 0.3648
Epoch [1/1], Step [64/126], Loss: 0.3580
Epoch [1/1], Step [80/126], Loss: 0.3292
Epoch [1/1], Step [96/126], Loss: 0.3356
Epoch [1/1], Step [112/126], Loss: 0.3293
Wall time: 1min 7s


Test predictions

In [37]:
test_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
print(next(iter(test_dataloader)).keys())
idx, X = next(enumerate(test_dataloader))

dict_keys(['idx', 'rgb', 'speed', 'steer', 'throttle', 'brake'])


In [38]:
data_ = normalize_img(X["rgb"].transpose(1,4)[:,:,:,:,0]).float().to(device)
target_ = (X["steer"],X["throttle"],X["brake"])

In [39]:
with torch.no_grad():
    outputs_ = net(data_)

In [40]:
print(outputs_[0])

tensor([[0.4432],
        [0.4883],
        [0.3209],
        [0.4083],
        [0.3334],
        [0.3575],
        [0.3554],
        [0.3668],
        [0.4578],
        [0.4050],
        [0.3630],
        [0.4245],
        [0.3495],
        [0.3716],
        [0.3939],
        [0.3768]], device='cuda:0')


In [41]:
print(target_[0])

tensor([[-0.0000],
        [-0.2180],
        [ 0.0000],
        [-0.0000],
        [ 0.0000],
        [ 0.0000],
        [-0.0000],
        [-0.0015],
        [-0.0000],
        [ 0.0045],
        [-0.0015],
        [-0.0005],
        [ 0.0000],
        [-0.0050],
        [-0.0000],
        [-0.0080]], dtype=torch.float64)


save & load

In [ ]:
torch.save(net, 'rgb_resnet.pth')

In [ ]:
net = torch.load('rgb_resnet.pth')

Time test

In [ ]:
# ohne preprocessing ca 16-17 sekunden. Mit preprocessing ca 35 sekunden
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
at = time.time()
for batch_idx, data in enumerate(train_dataloader):
    data_ = normalize_img(data["rgb"].transpose(1,4)[:,:,:,:,0]).float()
    target_ = torch.stack([data["speed"],data["steer"],data["throttle"],data["brake"]])[:,:,0].transpose(0,1).float()
et = time.time()
print(et-at)
